## Informações Gerais
 | Informações | Detalhes |
 |------------|-------------|
 |Nome Tabela | gold.venda_total_estabelecimento |
 |Origem | silver.pedidos |

## Histórico de Atualizações
 | Data | Desenvolvido por | Motivo |
 |:----:|--------------|--------|
 |14/09/2024 | Daniel Coutinho  | Criação do notebook |

In [0]:
%run /Projeto_Ampev/00.config/Configurações

In [0]:
# Importações
from pyspark.sql.functions import current_date, current_timestamp, expr

In [0]:
# Nome do esquema e tabela no catálogo
database = "gold"
tabela = "venda_total_estabelecimento"

In [0]:
# Funções

#Função que aplica os comentários na tabela
def adicionaComentariosTabela(a,b,c,d):
    spark.sql(f"COMMENT ON TABLE {a}.{b} IS '{c}'")
    for key,value in d.items():
        sqlaux = f"ALTER TABLE {a}.{b} CHANGE COLUMN {key} COMMENT '{value}'"
        spark.sql(sqlaux)

In [0]:
#criação do df_pedidos com dados necessários para responder as perguntas do negócio
df_pedidos = spark.sql(
f""" 
SELECT
    CAST(id_estabelecimento AS INT),
    nome_estabelecimento,
    produto AS nome_produto,
    CAST(SUM(quantidade) AS INT) AS quantidade_total,
    CAST(SUM(preco) AS DECIMAL(20,2)) AS valor_total
FROM
    silver.pedidos
GROUP BY all
""")

In [0]:
# incluir colunas de controle
df_pedidos = df_pedidos.withColumn("data_carga", current_date())
df_pedidos = df_pedidos.withColumn("data_hora_carga", expr("current_timestamp() - INTERVAL 3 HOURS"))

In [0]:
#Comentario Tabela

comentario_tabela = 'Entidade os produtos mais vendidos por estabelecimento'

lista_comentario_colunas = {
'id_estabelecimento' : 'Id do estabelecimento.',
'nome_estabelecimento' : 'Nome do estabelecimento.',
'nome_produto' : 'Nome do produto.',
'quantidade_total' : 'Quantidade total vendida.',
'valor_total' : 'Valor total vendido',
'data_carga' : 'Data que o registro foi carregado',
'data_hora_carga' : 'Data e hora que o registro foi carregado'
}

In [0]:
# Salvar o DataFrame no formato Delta
df_pedidos.write \
          .format('delta') \
          .mode('overwrite') \
          .clusterBy("id_estabelecimento") \
          .option('overwriteSchema', 'true') \
          .saveAsTable(f'{database}.{tabela}')
adicionaComentariosTabela(database, tabela, comentario_tabela, lista_comentario_colunas)
print("Dados gravados com sucesso!")

Dados gravados com sucesso!


In [0]:
# Otimização das tabelas
spark.sql(f"OPTIMIZE {database}.{tabela}")
print(f"Processo de otimização finalizado!.")

Processo de otimização finalizado!.


In [0]:
%sql
SELECT *
FROM gold.venda_total_estabelecimento

id_estabelecimento,nome_estabelecimento,nome_produto,quantidade_total,valor_total,data_carga,data_hora_carga
31,Bar e Grill,Água Mineral,470,17.43,2024-09-19,2024-09-19T19:51:26.9Z
31,Bar e Grill,Suco Natural,412,16.41,2024-09-19,2024-09-19T19:51:26.9Z
31,Bar e Grill,Vinho ABC,1020,49.30,2024-09-19,2024-09-19T19:51:26.9Z
34,Sorveteria Sorvetão,Suco Natural,484,3.52,2024-09-19,2024-09-19T19:51:26.9Z
34,Sorveteria Sorvetão,Cerveja XYZ,308,7.21,2024-09-19,2024-09-19T19:51:26.9Z
34,Sorveteria Sorvetão,Refrigerante 123,392,13.64,2024-09-19,2024-09-19T19:51:26.9Z
28,Padaria São João,Suco Natural,893,16.46,2024-09-19,2024-09-19T19:51:26.9Z
28,Padaria São João,Água Mineral,51,13.69,2024-09-19,2024-09-19T19:51:26.9Z
28,Padaria São João,Vinho ABC,349,1.26,2024-09-19,2024-09-19T19:51:26.9Z
28,Padaria São João,Refrigerante 123,496,28.79,2024-09-19,2024-09-19T19:51:26.9Z


In [0]:
%sql
--Quais são as empresas que mais compraram produtos? 
SELECT nome_estabelecimento
	,SUM(quantidade_total) AS total_vendido
FROM gold.venda_total_estabelecimento
GROUP BY nome_estabelecimento
ORDER BY total_vendido DESC LIMIT 5;

nome_estabelecimento,total_vendido
Padoca do Zé,3077
Farmácia Popular,2678
Barbearia do Toninho,2656
Padaria do Paulo,2361
Academia Fit,2227


In [0]:
%sql
--O estabelecimento que mais vendeu
SELECT nome_estabelecimento
	,SUM(quantidade_total) AS total_vendido
FROM gold.venda_total_estabelecimento
GROUP BY ALL
ORDER BY total_vendido DESC LIMIT 1;

nome_estabelecimento,total_vendido
Padoca do Zé,3077


In [0]:
%sql
--No estabelecimento que mais comprou, quais produtos mais vendidos?
WITH EstabelecimentoMaisVendeu
AS (
	SELECT nome_estabelecimento
		,SUM(quantidade_total) AS total_vendido
	FROM gold.venda_total_estabelecimento
	GROUP BY nome_estabelecimento
	ORDER BY total_vendido DESC LIMIT 1
	)
SELECT nome_produto
	,SUM(quantidade_total) AS total_vendido
FROM gold.venda_total_estabelecimento
WHERE nome_estabelecimento = (
		SELECT nome_estabelecimento
		FROM EstabelecimentoMaisVendeu
		)
GROUP BY ALL
ORDER BY total_vendido DESC

nome_produto,total_vendido
Água Mineral,990
Vinho ABC,880
Refrigerante 123,830
Cerveja XYZ,315
Suco Natural,62


Databricks visualization. Run in Databricks to view.